# FSL FEAT nipype

In [1]:
import os
import glob

import nipype
import nipype.interfaces.io as nio
import nipype.interfaces.fsl as fsl
import nipype.interfaces.ants as ants
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.algorithms.modelgen as model

230223-10:06:37,418 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.7.0


## pipeline setup

In [2]:
# Which dataset are we looking at?
# this_dataset = 'Leipzig_7T_SM'
# this_dataset = 'Leipzig_7T_GdH'
# this_dataset = 'NTNU_7T_SJSI'
this_dataset = 'aron_3T'

In [6]:
# general set-up
base_dir = '/home/scotti/projects/3t_7t_sst_comparison'
work_dir = os.path.join(base_dir, 'processing', 'nipype_workflow_folders')
roi_hp_dir = os.path.join(base_dir, 'derivatives', 'fsl_feat_roi_func', this_dataset)

hpcutoff = 10000.   # in seconds
model_n = [0]
spaces = ['MNI152NLin2009cAsym']   # shouldn't touch this but just in case we _do_ want to go back to MNI....

if this_dataset == 'Leipzig_7T_SM':
    t_r = 3.0
    subject_ids = [x.split('/')[-2].split('-')[-1] for x in glob.glob(os.path.join(roi_hp_dir, 'sub-*/'))]
    numsubs = len(subject_ids)
    task = 'stop'
elif this_dataset == 'Leipzig_7T_GdH':
    t_r = 2.0
    subject_ids = [x.split('/')[-1].split('-')[-1] for x in glob.glob(os.path.join(roi_hp_dir, 'sub-*T'))]
    numsubs = len(subject_ids)
    task = 'stop'
elif this_dataset == 'aron_3T':
    t_r = 2.0
    subject_ids = [x.split('/')[-2].split('-')[-1] for x in glob.glob(os.path.join(roi_hp_dir, 'sub-*/'))]
    numsubs = len(subject_ids)
    task = 'stopsignal'
elif this_dataset == 'NTNU_7T_SJSI':
    t_r = 1.38
#     numsubs = 20
    subject_ids = [x.split('/')[-2].split('-')[-1] for x in glob.glob(os.path.join(roi_hp_dir, 'sub-*/'))]
    numsubs = len(subject_ids)
    task = 'sst'
elif this_dataset == 'openfmri_3T':
    t_r = 2.0
#     numsubs = 97
    subject_ids = [x.split('/')[-2].split('-')[-1] for x in glob.glob(os.path.join(roi_hp_dir, 'sub-*/'))]
    numsubs = len(subject_ids)
    task = 'stopsignal'
    
# make sure all files required are found correctly    
rois_ = glob.glob(os.path.join(base_dir, 'derivatives', 'fsl_feat_roi_func', this_dataset,
                                        'sub-*', 'func',  f'sub-*_task-{task}_run-*_space-*_desc-preproc_bold.nii.gz'))
masks_ = glob.glob(os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset, 
                              'sub-*', 'func', f'sub-*_task-{task}_run-*_space-*_desc-brain_mask.nii.gz'))
# comp_ = glob.glob(os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset, 
#                                     'sub-*', 'anat', 'sub-*_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'))
# xfm_ = glob.glob(os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset,
#                               'sub-*', 'func',  f'sub-*_task-{task}_run-*_from-scanner_to-T1w_mode-image_xfm.txt'))
events_ = glob.glob(os.path.join(base_dir, f'derivatives/event_files/{this_dataset}/sub-*/func/sub-*run-*_events.tsv'))
    
template_brain = os.path.join(base_dir,'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')

In [7]:
print(f"""
this dataset : {this_dataset}
tr : {t_r}
number of subjects : {numsubs}
subject ids : {subject_ids}
roi_hp_dir : {roi_hp_dir}
model ns : {model_n}
no. rois : {len(rois_)}
no. masks : {len(masks_)}
no.events : {len(events_)}

""")


this dataset : aron_3T
tr : 2.0
number of subjects : 14
subject ids : ['15', '02', '14', '13', '10', '12', '05', '11', '06', '04', '01', '03', '07', '09']
roi_hp_dir : /home/scotti/projects/3t_7t_sst_comparison/derivatives/fsl_feat_roi_func/aron_3T
model ns : [0]
no. rois : 40
no. masks : 40
no.events : 40




# first & second level models

In [8]:
def get_session_info(subject_id, run, task='stop', this_dataset='Leipzig_7T_SM', space='T1w', shift=-1.38/2, model_n=0,
                    root_dir='/home/scotti/projects/3t_7t_sst_comparison', include_physio=True):
    # other space: MNI152NLin2009cAsym
    import pandas as pd
    import numpy as np
    import os
    from nipype.interfaces.base import Bunch
            
    ### files ###
    sub = subject_id
    event_fn = f'{root_dir}/derivatives/event_files/{this_dataset}/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_events.tsv'
    confounds_fn = f'{root_dir}/derivatives/fmriprep/{this_dataset}/fmriprep/fmriprep/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_desc-confounds_timeseries.tsv'

    events = pd.read_csv(event_fn, sep='\t', index_col=None)
    events['duration'] = 0.001  # stick functions
    if model_n == 0:
        events = events.loc[events.trial_type.isin(['ss', 'fs', 'go'])]
    elif model_n == 1:
        events = events.loc[events.trial_type.isin(['response_left', 'response_right'])]
    events = events[['onset', 'trial_type', 'duration']]

    # slice time correction, nb: shift should be a negative number for STC
    events['onset'] += shift  

    ### confounds ###
    confounds = pd.read_csv(confounds_fn, sep='\t')
#        cosine_cols = [x for x in confounds.columns if 'cos' in x]
    include_confounds = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'dvars', 'framewise_displacement'] #+ cosine_cols
    confounds = confounds[include_confounds].fillna(method='bfill')

    # get retroicor
    if include_physio:
        ## take first 20 aCompCor components
#         print("No retroicor found, including 20 a_comp_cor components")
        a_comp_cor = pd.read_csv(confounds_fn, sep='\t')[['a_comp_cor_' + str(x).zfill(2) for x in range(20)]]
        confounds = pd.concat([confounds, a_comp_cor], axis=1)

    # save the confounds that we actually include in the GLM to csv, no header or index
    confounds_fn = f'{root_dir}/derivatives/confounds/{this_dataset}/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_desc-confounds_timeseries.txt'
    os.makedirs(os.path.dirname(confounds_fn), exist_ok=True)
    confounds.to_csv(confounds_fn, sep='\t', header=False, index=False)

    ### Make bunch ###
    if model_n == 0:
        conditions=['fs',
                    'ss',
                    'go']

    elif model_n ==1:
        conditions = [
                     'response_left',
                     'response_right'
                     ]

    onsets = [events.loc[events.trial_type==trial_type,'onset'].tolist() for trial_type in conditions]
    durations = [events.loc[events.trial_type==trial_type,'duration'].tolist() for trial_type in conditions]
    amplitudes = [[1] * events.loc[events.trial_type==trial_type,'onset'].shape[0] for trial_type in conditions]
#             amplitudes = [events.loc[events.trial_type==trial_type,'modulation'].tolist() for trial_type in conditions]

    info = Bunch(conditions=conditions,
                 onsets=onsets,
                 durations=durations,
                 amplitudes=amplitudes)
#         print(info)
    
    # always return info and the confounds_fn
    return info, confounds_fn #, contrasts[0]

if model_n == [0]:
    contrasts = [('fs', 'T', ['fs'], [1.0]),
                 ('ss', 'T', ['ss'], [1.0]),
                 ('go', 'T', ['go'], [1.0]),
                 ('fs-go', 'T', ['fs', 'go'], [1, -1]),
                 ('fs-ss', 'T', ['fs', 'ss'], [1, -1]),
                 ('ss-go', 'T', ['ss', 'go'], [1, -1])
                 ]
    
elif model_n == [1]:
    contrasts = [('response_left', 'T', ['response_left'], [1.0]),
                 ('response_right', 'T', ['response_right'], [1.0]),
                 ('left-right', 'T', ['response_left','response_right'], [1,-1])
                 ]

In [9]:
def get_runs_per_sub(subject_id):
    # here we can add the runs per subject if some subject miss a run or two
    if not subject_id in ['11','12']:
        runs = [1,2,3]
    else:
        runs = [1,2]
    
    return runs

In [10]:
workflow = pe.Workflow(name='feat_level12_sst_roi')
workflow.base_dir = os.path.join(base_dir, 'processing', 'nipype_workflow_folders', this_dataset)
workflow.config = {"execution": {"crashdump_dir":os.path.join(base_dir, 'processing', 'crashdumps')}}

# identity
identity = pe.Node(util.IdentityInterface(fields=['subject_id', 'space', 'model_n']), name='identity')
identity.iterables = [('subject_id', subject_ids),
                      ('space', spaces),
                      ('model_n', model_n)]

# selector
# if running only one run, copy the mask from run 1 to run 2, 'mask' variable below needs a list, it will crash if only one is found
#(terrible way to get around this.. fix at some point.. lazy..)
templates = {'roi_funcs': os.path.join(base_dir, 'derivatives', 'fsl_feat_roi_func', this_dataset,
                                            'sub-{subject_id}', 'func',  
                                            f'sub-{{subject_id}}_task-{task}_run-*_space-{{space}}*_desc-preproc_bold.nii.gz'),
             'mask': os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset, 
                                  'sub-{subject_id}', 'func', 
                                  f'sub-{{subject_id}}_task-{task}_run-*_space-{{space}}_desc-brain_mask.nii.gz')}

selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')
workflow.connect(identity, 'space', selector, 'space')

# get run info per sub
run_info_getter = pe.Node(util.Function(function=get_runs_per_sub,
                                        input_names=['subject_id'],
                                        output_names=['runs']), name='run_info_getter')
workflow.connect(identity, 'subject_id', run_info_getter, 'subject_id')

# session info getter
session_info_getter = pe.MapNode(util.Function(function=get_session_info,
                                     input_names=['subject_id', 'run', 'task', 'this_dataset', 'space', 'shift', 'model_n'],
                                     output_names=['session_info', 'confounds']),
                                 iterfield=['run'],
                                 name='session_info_getter')
session_info_getter.inputs.task=task
session_info_getter.inputs.this_dataset=this_dataset
session_info_getter.inputs.space='MNI152NLin2009cAsym'
session_info_getter.inputs.shift = -t_r/2

workflow.connect(identity, 'subject_id', session_info_getter, 'subject_id')
workflow.connect(run_info_getter, 'runs', session_info_getter, 'run')
workflow.connect(identity, 'model_n', session_info_getter, 'model_n')


# model setup
specifymodel = pe.Node(model.SpecifyModel(), name='specifymodel1')
specifymodel.inputs.input_units = 'secs'
specifymodel.inputs.time_repetition = t_r
specifymodel.inputs.high_pass_filter_cutoff = hpcutoff

workflow.connect(session_info_getter, 'session_info', specifymodel, 'subject_info')
## old flow (with cosines): immediately connect to specifymodel
workflow.connect(selector, 'roi_funcs', specifymodel, 'functional_runs')

# Level 1 design
level1design = pe.Node(interface=fsl.Level1Design(), name="level1design")
level1design.inputs.interscan_interval = t_r
level1design.inputs.bases = {'dgamma': {'derivs': True}}
level1design.inputs.contrasts = contrasts
level1design.inputs.model_serial_correlations = True

workflow.connect(specifymodel, 'session_info', level1design, 'session_info')
# workflow.connect(session_info_getter, 'contrasts', level1design, 'contrasts')

# FEAT model
modelgen = pe.MapNode(interface=fsl.FEATModel(), iterfield=['ev_files', 'fsf_file', 'args'], name='modelgen')

workflow.connect(level1design, 'ev_files', modelgen, 'ev_files')
workflow.connect(level1design, 'fsf_files', modelgen, 'fsf_file')
workflow.connect(session_info_getter, 'confounds', modelgen, 'args')   # add confounds here


# FILM GLS
iterfield = ['design_file', 'in_file', 'tcon_file']
modelestimate = pe.MapNode(interface=fsl.FILMGLS(smooth_autocorr=False, # prewhitening yes or no?
#                                                  autocorr_noestimate=True,
                                                 mask_size = 0,
                                                 threshold = 100),  # threshold for signal, default = 1000 but will lose data that way as 1000 is higher than alot of the voxel signals
                                                 name='modelestimate',
                                                 iterfield=iterfield,
                                                 mem_gb=10,
                                                 full_data = True,
                                                 output_pwdata=True)

# ## old flow (with cosines): immediately connect to modelestimate
workflow.connect(selector, 'roi_funcs', modelestimate, 'in_file')
# # new flow: connect high-passed data
# workflow.connect(highpass, 'out_file', modelestimate, 'in_file')


workflow.connect(modelgen, 'design_file', modelestimate, 'design_file')
workflow.connect(modelgen, 'con_file', modelestimate, 'tcon_file')


#### Fixed effects
# merge copes, varcopes
copemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="copemerge")

varcopemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="varcopemerge")

maskemerge = pe.MapNode(interface=fsl.Merge(dimension='t'),
                       iterfield=['in_files'],
                       name="maskemerge")

def sort_copes(files):
    numelements = len(files[0])
    outfiles = []
    for i in range(numelements):
        outfiles.insert(i,[])
        for j, elements in enumerate(files):
            outfiles[i].append(elements[i])
    return outfiles

workflow.connect(modelestimate, ('copes',sort_copes), copemerge, 'in_files')
workflow.connect(modelestimate, ('varcopes',sort_copes), varcopemerge, 'in_files')


level2model = pe.Node(interface=fsl.L2Model(), name='l2model')
def num_copes(files):
    return len(files)
workflow.connect(modelestimate, ('copes',num_copes), level2model, 'num_copes')


pickfirst = lambda x: x[0]
flameo = pe.MapNode(
    interface=fsl.FLAMEO(run_mode='fe'),
    name="flameo",
    iterfield=['cope_file', 'var_cope_file'])

workflow.connect([
    (selector, flameo, [(('mask', pickfirst), 'mask_file')]),
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level2model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

# ## Warp to MNI-space
# def warp_files(copes, varcopes, zstats, tdofs, mat, template_brain):
#     # stolen from https://dpaniukov.github.io/2016/07/14/three-level-analysis-with-fsl-and-ants-2.html
#     import nipype.interfaces.ants as ants

#     out_copes = []
#     out_varcopes = []
#     out_zstats = []
#     out_tdofs = []
    
#     warp = ants.ApplyTransforms()
#     warp.inputs.input_image_type = 0
#     warp.inputs.interpolation = 'Linear'
#     warp.inputs.invert_transform_flags = [False] #,False]
#     warp.inputs.reference_image = template_brain
#     warp.inputs.transforms = mat

#     if not isinstance(copes, list):
#         copes = [copes]
#         varcopes = [varcopes]
#         zstats = [zstats]
#         tdofs = [tdofs]
    
#     for cope in copes:
#         warp.inputs.input_image = cope
#         res=warp.run()
#         out_copes.append(str(res.outputs.output_image))

#     for varcope in varcopes:
#         warp.inputs.input_image = varcope
#         res=warp.run()
#         out_varcopes.append(str(res.outputs.output_image))
        
#     for zstat in zstats:
#         warp.inputs.input_image = zstat
#         res=warp.run()
#         out_zstats.append(str(res.outputs.output_image))
    
#     for tdof in tdofs:
#         warp.inputs.input_image = tdof
#         res=warp.run()
#         out_tdofs.append(str(res.outputs.output_image))

#     return out_copes, out_varcopes, out_zstats, out_tdofs

# warpfunc = pe.MapNode(util.Function(input_names=['copes', 'varcopes', 'zstats', 'tdofs', 'mat','template_brain'],
#                                output_names=['out_copes', 'out_varcopes', 'out_zstats', 'out_tdofs'],
#                                function=warp_files),
#                                iterfield=['copes', 'varcopes', 'zstats', 'tdofs'],
#                   name='warpfunc')

# warpfunc.inputs.template_brain = template_brain
# workflow.connect(flameo, 'copes', warpfunc, 'copes')
# workflow.connect(flameo, 'var_copes', warpfunc, 'varcopes')
# workflow.connect(flameo, 'zstats', warpfunc, 'zstats')
# workflow.connect(flameo, 'tdof', warpfunc, 'tdofs')
# workflow.connect(selector, 'composite', warpfunc, 'mat')

#### post-FE analyis
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')

workflow.connect(selector, ('mask', pickfirst), smoothestimate, 'mask_file')
workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

#
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
workflow.connect(selector, ('mask', pickfirst), get_volume, 'in_file')

# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])] 
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

def convert_volume(input):
    return int(input[1])

workflow.connect(get_volume, ('out_stat', convert_volume), grf_cluster, 'volume')
grf_cluster.inputs.out_threshold_file = True


## datasink
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(base_dir, 'derivatives', 'glm_feat_sst_roi')

#_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_002
substitutions = [(f'mni/level2_{stat_type}s/_model_n_{model_n_}_space_MNI152NLin2009cAsym_subject_id_{sub}/_flameo{contrast_n}/{stat_type}1.nii.gz',
                  f'{this_dataset}/sub-{sub}/func/model-{model_n_}/sub-{sub}_task-{task}_space-MNI152NLin2009cAsym_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
                  for sub in subject_ids
                  for contrast_n in range(len(contrasts))
                  for model_n_ in model_n
                  for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
                  ]


# substitutions = [(f'mni/level2_{stat_type}s/_model_n_{model_n_}_space_MNI152NLin2009cAsym_subject_id_{sub}/_warpfunc{contrast_n}/{stat_type}1_trans.nii.gz',
#                    f'{this_dataset}/sub-{sub}/func/model-{model_n_}/sub-{sub}_task-{task}_space-MNI152NLin2009cAsym_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
#                   for sub in subject_ids
#                   for contrast_n in range(len(contrasts))
#                   for model_n_ in model_n
#                   for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
#                   ]

ds.inputs.substitutions = substitutions


## cluster thresholds: leave for now, this is more of a third-level model thing anyway
# workflow.connect(grf_cluster, 'threshold_file', ds, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'locnalmax_txt_file', ds, 'grf_localmax_txt_file')

## T1w-space
workflow.connect(flameo, 'zstats', ds, 'subject_level_model.mni.level2_zstats')
workflow.connect(flameo, 'copes', ds, 'subject_level_model.mni.level2_copes')
workflow.connect(flameo, 'var_copes', ds, 'subject_level_model.mni.level2_varcopes')
workflow.connect(flameo, 'tdof', ds, 'subject_level_model.mni.level2_tdof_ts')

# ## MNI-space
# workflow.connect(warpfunc, 'out_zstats', ds, 'subject_level_model.mni.level2_zstats')
# workflow.connect(warpfunc, 'out_copes', ds, 'subject_level_model.mni.level2_copes')
# workflow.connect(warpfunc, 'out_varcopes', ds, 'subject_level_model.mni.level2_varcopes')
# workflow.connect(warpfunc, 'out_tdofs', ds, 'subject_level_model.mni.level2_tdof_ts')
# workflow.connect(flameo, 'tdof', ds, 'level2_tdof')

In [11]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs':20, 'memory_gb': 200})

230223-10:10:00,135 nipype.workflow INFO:
	 Workflow feat_level12_sst_roi settings: ['check', 'execution', 'logging', 'monitoring']
230223-10:10:00,464 nipype.workflow INFO:
	 Running in parallel.
230223-10:10:00,939 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 28 jobs ready. Free memory (GB): 200.00/200.00, Free processors: 20/20.
230223-10:10:01,32 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_15/selector".
230223-10:10:01,33 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.run_info_getter" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_15/run_info_getter".
230223-10:10:01,33 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.selector" in "/home/scott

230223-10:10:01,44 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.000419s.
230223-10:10:01,44 nipype.workflow INFO:
	 [Node] Executing "run_info_getter" <nipype.interfaces.utility.wrappers.Function>
230223-10:10:01,44 nipype.workflow INFO:
	 [Node] Executing "selector" <nipype.interfaces.io.SelectFiles>
230223-10:10:01,45 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.000464s.
230223-10:10:01,43 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.0006s.
230223-10:10:01,45 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.001217s.
230223-10:10:01,45 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.001235s.
230223-10:10:01,45 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.000544s.
230223-10:10:01,45 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.000626s.
230223-10:10:01,46 nipype.workflow INFO:
	 [Node] Finished "run_info_get

230223-10:10:03,7 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.get_volume" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_02/get_volume".
230223-10:10:03,11 nipype.workflow INFO:
	 [Node] Executing "get_volume" <nipype.interfaces.fsl.utils.ImageStats>
230223-10:10:03,14 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.get_volume" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_14/get_volume".
230223-10:10:03,17 nipype.workflow INFO:
	 [Node] Executing "get_volume" <nipype.interfaces.fsl.utils.ImageStats>
230223-10:10:03,18 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.get_volume" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin

230223-10:10:05,20 nipype.workflow INFO:
	 [Node] Executing "_session_info_getter2" <nipype.interfaces.utility.wrappers.Function>
230223-10:10:05,21 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter2" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_02/session_info_getter/mapflow/_session_info_getter2".
230223-10:10:05,21 nipype.workflow INFO:
	 [Node] Executing "_session_info_getter0" <nipype.interfaces.utility.wrappers.Function>
230223-10:10:05,22 nipype.workflow INFO:
	 [Node] Executing "_session_info_getter1" <nipype.interfaces.utility.wrappers.Function>
230223-10:10:05,24 nipype.workflow INFO:
	 [Node] Executing "_session_info_getter2" <nipype.interfaces.utility.wrappers.Function>
230223-10:10:06,530 nipype.workflow INFO:
	 [Node] Finished "_session_info_getter2", elapsed time 1.50878s.
230223-10:10:06,548 nipype.workflow INFO:
	 [Node] Finished "_session_inf

230223-10:10:08,931 nipype.workflow INFO:
	 [Job 31] Completed (feat_level12_sst_roi.session_info_getter).
230223-10:10:08,932 nipype.workflow INFO:
	 [Job 230] Completed (_session_info_getter0).
230223-10:10:08,933 nipype.workflow INFO:
	 [Job 231] Completed (_session_info_getter1).
230223-10:10:08,933 nipype.workflow INFO:
	 [Job 232] Completed (_session_info_getter2).
230223-10:10:08,934 nipype.workflow INFO:
	 [Job 233] Completed (_session_info_getter0).
230223-10:10:08,935 nipype.workflow INFO:
	 [MultiProc] Running 14 tasks, and 33 jobs ready. Free memory (GB): 197.20/200.00, Free processors: 6/20.
                     Currently running:
                       * feat_level12_sst_roi.get_volume
                       * feat_level12_sst_roi.get_volume
                       * feat_level12_sst_roi.get_volume
                       * feat_level12_sst_roi.get_volume
                       * feat_level12_sst_roi.get_volume
                       * feat_level12_sst_roi.get_volume
      

...

230223-10:24:07,836 nipype.workflow INFO:
	 [Node] "_grf_cluster0" found cached.
230223-10:24:07,835 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_07/_threshold_3.1/grf_cluster/mapflow/_grf_cluster0".
230223-10:24:07,837 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_07/_threshold_2.3/grf_cluster/mapflow/_grf_cluster1".
230223-10:24:07,837 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster0" - collecting precomputed outputs
230223-10:24:07,838 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster1" - collecting precomputed outputs
230223-10:24:07,838 nipype.workflow INFO:
	 [Node] "_grf_cluster0" found cached.
230223-10:24:07,839 nipype.workflow INFO:


230223-10:24:09,833 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate4" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_01/smoothestimate/mapflow/_smoothestimate4".
230223-10:24:09,834 nipype.workflow INFO:
	 [Node] Cached "_smoothestimate4" - collecting precomputed outputs
230223-10:24:09,835 nipype.workflow INFO:
	 [Node] "_smoothestimate4" found cached.
230223-10:24:09,836 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate5" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_01/smoothestimate/mapflow/_smoothestimate5".
230223-10:24:09,837 nipype.workflow INFO:
	 [Node] Cached "_smoothestimate5" - collecting precomputed outputs
230223-10:24:09,837 nipype.workflow INFO:
	 [Node] "_smoothestimate5" found cached.
230223-10:24:10,315 nipype.workflow INFO:


230223-10:24:13,827 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster5" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_01/_threshold_3.1/grf_cluster/mapflow/_grf_cluster5".
230223-10:24:13,827 nipype.workflow INFO:
	 [Node] Cached "_smoothestimate3" - collecting precomputed outputs
230223-10:24:13,827 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster2" <nipype.interfaces.fsl.model.Cluster>
230223-10:24:13,828 nipype.workflow INFO:
	 [Node] "_smoothestimate3" found cached.
230223-10:24:13,828 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster4" <nipype.interfaces.fsl.model.Cluster>
230223-10:24:13,829 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster3" <nipype.interfaces.fsl.model.Cluster>
230223-10:24:13,829 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate4" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T

230223-10:24:15,840 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster4" - collecting precomputed outputs
230223-10:24:15,840 nipype.workflow INFO:
	 [Node] "_grf_cluster4" found cached.
230223-10:24:15,840 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster3" - collecting precomputed outputs
230223-10:24:15,841 nipype.workflow INFO:
	 [Node] "_grf_cluster3" found cached.
230223-10:24:15,842 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster5" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_01/_threshold_2.3/grf_cluster/mapflow/_grf_cluster5".
230223-10:24:15,842 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster4" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_01/_threshold_3.1/grf_cluster/mapflow/_grf_cluster4".
230223-10:24:15,843 nipype.workflow INFO:


230223-10:24:19,760 nipype.workflow INFO:
	 [Job 836] Completed (_grf_cluster0).
230223-10:24:19,761 nipype.workflow INFO:
	 [Job 837] Completed (_grf_cluster1).
230223-10:24:19,761 nipype.workflow INFO:
	 [Job 838] Completed (_grf_cluster2).
230223-10:24:19,762 nipype.workflow INFO:
	 [Job 839] Completed (_grf_cluster3).
230223-10:24:19,763 nipype.workflow INFO:
	 [Job 840] Completed (_grf_cluster4).
230223-10:24:19,763 nipype.workflow INFO:
	 [Job 841] Completed (_grf_cluster5).
230223-10:24:19,764 nipype.workflow INFO:
	 [Job 842] Completed (_grf_cluster0).
230223-10:24:19,764 nipype.workflow INFO:
	 [Job 843] Completed (_grf_cluster1).
230223-10:24:19,765 nipype.workflow INFO:
	 [Job 844] Completed (_grf_cluster2).
230223-10:24:19,765 nipype.workflow INFO:
	 [Job 845] Completed (_grf_cluster3).
230223-10:24:19,766 nipype.workflow INFO:
	 [Job 846] Completed (_grf_cluster4).
230223-10:24:19,766 nipype.workflow INFO:
	 [Job 847] Completed (_grf_cluster5).
230223-10:24:19,769 nipype.w

In [9]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/crashdumps/crash-20230202-194255-scotti-_modelestimate0-569bdeac-24e9-4591-a258-3f0682a4dc27.pklz')
res

{'node': _modelestimate0,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 750, in _run_command\n    raise NodeExecutionError(\n',
  'nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node _modelestimate0.\n\nTraceback (most recent call last):\n  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/si

In [11]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_10/modelestimate/mapflow/_modelestimate1/_inputs.pklz')
res

{'threshold': 1000.0,
 'tcon_file': '/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_10/modelgen/mapflow/_modelgen1/run1.con',
 'in_file': '/home/scotti/projects/3t_7t_sst_comparison/derivatives/fsl_feat_roi_func/aron_3T/sub-10/func/sub-10_task-stopsignal_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
 'design_file': '/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_10/modelgen/mapflow/_modelgen1/run1.mat',
 'smooth_autocorr': False,
 'mask_size': 0,
 'autocorr_noestimate': True,
 'results_dir': 'results',
 'output_type': 'NIFTI_GZ',
 'environ': {'FSLOUTPUTTYPE': 'NIFTI_GZ'}}

In [12]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_3p125_space_T1w_subject_id_01/modelestimate/mapflow/_modelestimate1/_inputs.pklz')
res

{'threshold': 1000.0,
 'tcon_file': '/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_3p125_space_T1w_subject_id_01/modelgen/mapflow/_modelgen1/run1.con',
 'in_file': '/home/scotti/projects/3t_7t_sst_comparison/derivatives/susan_smoothed_hp_func/aron_3T/sub-01/func/sub-01_task-stopsignal_run-2_space-T1w_desc-preproc_bold_smoothed_fwhm-3p125_tempfilt.nii.gz',
 'design_file': '/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_3p125_space_T1w_subject_id_01/modelgen/mapflow/_modelgen1/run1.mat',
 'smooth_autocorr': True,
 'mask_size': 5,
 'results_dir': 'results',
 'output_type': 'NIFTI_GZ',
 'environ': {'FSLOUTPUTTYPE': 'NIFTI_GZ'}}

In [ ]:
workflow = pe.Workflow(name='feat_level12_sst_roi')
workflow.base_dir = os.path.join(base_dir, 'processing', 'nipype_workflow_folders', this_dataset)
workflow.config = {"execution": {"crashdump_dir":os.path.join(base_dir, 'processing', 'crashdumps')}}

# identity
identity = pe.Node(util.IdentityInterface(fields=['subject_id', 'space', 'model_n']), name='identity')
identity.iterables = [('subject_id', subject_ids),
                      ('space', spaces),
                      ('model_n', model_n)]

# selector
# if running only one run, copy the mask from run 1 to run 2, 'mask' variable below needs a list, it will crash if only one is found
#(terrible way to get around this.. fix at some point.. lazy..)
templates = {'roi_funcs': os.path.join(base_dir, 'derivatives', 'fsl_feat_roi_func', this_dataset,
                                            'sub-{subject_id}', 'func',  
                                            f'sub-{{subject_id}}_task-{task}_run-*_space-{{space}}*_desc-preproc_bold.nii.gz'),
             'mask': os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset, 
                                  'sub-{subject_id}', 'func', 
                                  f'sub-{{subject_id}}_task-{task}_run-*_space-{{space}}_desc-brain_mask.nii.gz')}

selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')
workflow.connect(identity, 'space', selector, 'space')

# get run info per sub
run_info_getter = pe.Node(util.Function(function=get_runs_per_sub,
                                        input_names=['subject_id'],
                                        output_names=['runs']), name='run_info_getter')
workflow.connect(identity, 'subject_id', run_info_getter, 'subject_id')

# session info getter
session_info_getter = pe.MapNode(util.Function(function=get_session_info,
                                     input_names=['subject_id', 'run', 'task', 'this_dataset', 'space', 'shift', 'model_n'],
                                     output_names=['session_info', 'confounds']),
                                 iterfield=['run'],
                                 name='session_info_getter')
session_info_getter.inputs.task=task
session_info_getter.inputs.this_dataset=this_dataset
session_info_getter.inputs.space='MNI152NLin2009cAsym'
session_info_getter.inputs.shift = -t_r/2

workflow.connect(identity, 'subject_id', session_info_getter, 'subject_id')
workflow.connect(run_info_getter, 'runs', session_info_getter, 'run')
workflow.connect(identity, 'model_n', session_info_getter, 'model_n')


# model setup
specifymodel = pe.Node(model.SpecifyModel(), name='specifymodel1')
specifymodel.inputs.input_units = 'secs'
specifymodel.inputs.time_repetition = t_r
specifymodel.inputs.high_pass_filter_cutoff = hpcutoff

workflow.connect(session_info_getter, 'session_info', specifymodel, 'subject_info')
## old flow (with cosines): immediately connect to specifymodel
workflow.connect(selector, 'roi_funcs', specifymodel, 'functional_runs')

# Level 1 design
level1design = pe.Node(interface=fsl.Level1Design(), name="level1design")
level1design.inputs.interscan_interval = t_r
level1design.inputs.bases = {'dgamma': {'derivs': True}}
level1design.inputs.contrasts = contrasts
level1design.inputs.model_serial_correlations = True

workflow.connect(specifymodel, 'session_info', level1design, 'session_info')
# workflow.connect(session_info_getter, 'contrasts', level1design, 'contrasts')

# FEAT model
modelgen = pe.MapNode(interface=fsl.FEATModel(), iterfield=['ev_files', 'fsf_file', 'args'], name='modelgen')

workflow.connect(level1design, 'ev_files', modelgen, 'ev_files')
workflow.connect(level1design, 'fsf_files', modelgen, 'fsf_file')
workflow.connect(session_info_getter, 'confounds', modelgen, 'args')   # add confounds here


# FILM GLS
iterfield = ['design_file', 'in_file', 'tcon_file']
modelestimate = pe.MapNode(interface=fsl.FILMGLS(#smooth_autocorr=False, # prewhitening yes or no?
                                                 autocorr_noestimate=True,
                                                 mask_size = 0,
                                                 threshold = 1000),  # what to scale all inputs up to?
                                                 name='modelestimate',
                                                 iterfield=iterfield,
                                                 mem_gb=10)#,
                                                 #full_data = True)

# ## old flow (with cosines): immediately connect to modelestimate
workflow.connect(selector, 'roi_funcs', modelestimate, 'in_file')
# # new flow: connect high-passed data
# workflow.connect(highpass, 'out_file', modelestimate, 'in_file')


workflow.connect(modelgen, 'design_file', modelestimate, 'design_file')
workflow.connect(modelgen, 'con_file', modelestimate, 'tcon_file')


#### Fixed effects
# merge copes, varcopes
copemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="copemerge")

varcopemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="varcopemerge")

maskemerge = pe.MapNode(interface=fsl.Merge(dimension='t'),
                       iterfield=['in_files'],
                       name="maskemerge")

def sort_copes(files):
    numelements = len(files[0])
    outfiles = []
    for i in range(numelements):
        outfiles.insert(i,[])
        for j, elements in enumerate(files):
            outfiles[i].append(elements[i])
    return outfiles

workflow.connect(modelestimate, ('copes',sort_copes), copemerge, 'in_files')
workflow.connect(modelestimate, ('varcopes',sort_copes), varcopemerge, 'in_files')


level2model = pe.Node(interface=fsl.L2Model(), name='l2model')
def num_copes(files):
    return len(files)
workflow.connect(modelestimate, ('copes',num_copes), level2model, 'num_copes')


pickfirst = lambda x: x[0]
flameo = pe.MapNode(
    interface=fsl.FLAMEO(run_mode='fe'),
    name="flameo",
    iterfield=['cope_file', 'var_cope_file'])

workflow.connect([
    (selector, flameo, [(('mask', pickfirst), 'mask_file')]),
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level2model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

# ## Warp to MNI-space
# def warp_files(copes, varcopes, zstats, tdofs, mat, template_brain):
#     # stolen from https://dpaniukov.github.io/2016/07/14/three-level-analysis-with-fsl-and-ants-2.html
#     import nipype.interfaces.ants as ants

#     out_copes = []
#     out_varcopes = []
#     out_zstats = []
#     out_tdofs = []
    
#     warp = ants.ApplyTransforms()
#     warp.inputs.input_image_type = 0
#     warp.inputs.interpolation = 'Linear'
#     warp.inputs.invert_transform_flags = [False] #,False]
#     warp.inputs.reference_image = template_brain
#     warp.inputs.transforms = mat

#     if not isinstance(copes, list):
#         copes = [copes]
#         varcopes = [varcopes]
#         zstats = [zstats]
#         tdofs = [tdofs]
    
#     for cope in copes:
#         warp.inputs.input_image = cope
#         res=warp.run()
#         out_copes.append(str(res.outputs.output_image))

#     for varcope in varcopes:
#         warp.inputs.input_image = varcope
#         res=warp.run()
#         out_varcopes.append(str(res.outputs.output_image))
        
#     for zstat in zstats:
#         warp.inputs.input_image = zstat
#         res=warp.run()
#         out_zstats.append(str(res.outputs.output_image))
    
#     for tdof in tdofs:
#         warp.inputs.input_image = tdof
#         res=warp.run()
#         out_tdofs.append(str(res.outputs.output_image))

#     return out_copes, out_varcopes, out_zstats, out_tdofs

# warpfunc = pe.MapNode(util.Function(input_names=['copes', 'varcopes', 'zstats', 'tdofs', 'mat','template_brain'],
#                                output_names=['out_copes', 'out_varcopes', 'out_zstats', 'out_tdofs'],
#                                function=warp_files),
#                                iterfield=['copes', 'varcopes', 'zstats', 'tdofs'],
#                   name='warpfunc')

# warpfunc.inputs.template_brain = template_brain
# workflow.connect(flameo, 'copes', warpfunc, 'copes')
# workflow.connect(flameo, 'var_copes', warpfunc, 'varcopes')
# workflow.connect(flameo, 'zstats', warpfunc, 'zstats')
# workflow.connect(flameo, 'tdof', warpfunc, 'tdofs')
# workflow.connect(selector, 'composite', warpfunc, 'mat')

#### post-FE analyis
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')

workflow.connect(selector, ('mask', pickfirst), smoothestimate, 'mask_file')
workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

#
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
workflow.connect(selector, ('mask', pickfirst), get_volume, 'in_file')

# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])] 
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

def convert_volume(input):
    return int(input[1])

workflow.connect(get_volume, ('out_stat', convert_volume), grf_cluster, 'volume')
grf_cluster.inputs.out_threshold_file = True


## datasink
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(base_dir, 'derivatives', 'glm_feat_sst_roi')

#_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_002
substitutions = [(f'mni/level2_{stat_type}s/_model_n_{model_n_}_space_MNI152NLin2009cAsym_subject_id_{sub}/_flameo{contrast_n}/{stat_type}1.nii.gz',
                  f'{this_dataset}/sub-{sub}/func/model-{model_n_}/sub-{sub}_task-{task}_space-MNI152NLin2009cAsym_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
                  for sub in subject_ids
                  for contrast_n in range(len(contrasts))
                  for model_n_ in model_n
                  for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
                  ]


# substitutions = [(f'mni/level2_{stat_type}s/_model_n_{model_n_}_space_MNI152NLin2009cAsym_subject_id_{sub}/_warpfunc{contrast_n}/{stat_type}1_trans.nii.gz',
#                    f'{this_dataset}/sub-{sub}/func/model-{model_n_}/sub-{sub}_task-{task}_space-MNI152NLin2009cAsym_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
#                   for sub in subject_ids
#                   for contrast_n in range(len(contrasts))
#                   for model_n_ in model_n
#                   for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
#                   ]

ds.inputs.substitutions = substitutions


## cluster thresholds: leave for now, this is more of a third-level model thing anyway
# workflow.connect(grf_cluster, 'threshold_file', ds, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'locnalmax_txt_file', ds, 'grf_localmax_txt_file')

## T1w-space
workflow.connect(flameo, 'zstats', ds, 'subject_level_model.mni.level2_zstats')
workflow.connect(flameo, 'copes', ds, 'subject_level_model.mni.level2_copes')
workflow.connect(flameo, 'var_copes', ds, 'subject_level_model.mni.level2_varcopes')
workflow.connect(flameo, 'tdof', ds, 'subject_level_model.mni.level2_tdof_ts')

# ## MNI-space
# workflow.connect(warpfunc, 'out_zstats', ds, 'subject_level_model.mni.level2_zstats')
# workflow.connect(warpfunc, 'out_copes', ds, 'subject_level_model.mni.level2_copes')
# workflow.connect(warpfunc, 'out_varcopes', ds, 'subject_level_model.mni.level2_varcopes')
# workflow.connect(warpfunc, 'out_tdofs', ds, 'subject_level_model.mni.level2_tdof_ts')
# workflow.connect(flameo, 'tdof', ds, 'level2_tdof')

## when we were not using confounds

In [5]:
def get_session_info(subject_id, run, task='stop', this_dataset='Leipzig_7T_SM', space='T1w', shift=-1.38/2, model_n=0,
                    root_dir='/home/scotti/projects/3t_7t_sst_comparison', include_physio=True):
    # other space: MNI152NLin2009cAsym
    import pandas as pd
    import numpy as np
    import os
    from nipype.interfaces.base import Bunch
            
    ### files ###
    sub = subject_id
    event_fn = f'{root_dir}/derivatives/event_files/{this_dataset}/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_events.tsv'
#     confounds_fn = f'{root_dir}/derivatives/fmriprep/{this_dataset}/fmriprep/fmriprep/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_desc-confounds_timeseries.tsv'

    events = pd.read_csv(event_fn, sep='\t', index_col=None)
    events['duration'] = 0.001  # stick functions
    if model_n == 0:
        events = events.loc[events.trial_type.isin(['ss', 'fs', 'go'])]
    elif model_n == 1:
        events = events.loc[events.trial_type.isin(['response_left', 'response_right'])]
    events = events[['onset', 'trial_type', 'duration']]

    # slice time correction, nb: shift should be a negative number for STC
    events['onset'] += shift  

#     ### confounds ###
#     confounds = pd.read_csv(confounds_fn, sep='\t')
# #        cosine_cols = [x for x in confounds.columns if 'cos' in x]
#     include_confounds = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'dvars', 'framewise_displacement'] #+ cosine_cols
#     confounds = confounds[include_confounds].fillna(method='bfill')

#     # get retroicor
#     if include_physio:
#         ## take first 20 aCompCor components
# #         print("No retroicor found, including 20 a_comp_cor components")
#         a_comp_cor = pd.read_csv(confounds_fn, sep='\t')[['a_comp_cor_' + str(x).zfill(2) for x in range(20)]]
#         confounds = pd.concat([confounds, a_comp_cor], axis=1)

#     # save the confounds that we actually include in the GLM to csv, no header or index
#     confounds_fn = f'{root_dir}/derivatives/confounds/{this_dataset}/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_desc-confounds_timeseries.txt'
#     os.makedirs(os.path.dirname(confounds_fn), exist_ok=True)
#     confounds.to_csv(confounds_fn, sep='\t', header=False, index=False)

    ### Make bunch ###
    if model_n == 0:
        conditions=['fs',
                    'ss',
                    'go']

    elif model_n ==1:
        conditions = [
                     'response_left',
                     'response_right'
                     ]

    onsets = [events.loc[events.trial_type==trial_type,'onset'].tolist() for trial_type in conditions]
    durations = [events.loc[events.trial_type==trial_type,'duration'].tolist() for trial_type in conditions]
    amplitudes = [[1] * events.loc[events.trial_type==trial_type,'onset'].shape[0] for trial_type in conditions]
#             amplitudes = [events.loc[events.trial_type==trial_type,'modulation'].tolist() for trial_type in conditions]

    info = Bunch(conditions=conditions,
                 onsets=onsets,
                 durations=durations,
                 amplitudes=amplitudes)
#         print(info)
    
    # always return info and the confounds_fn
    return info #, confounds_fn #, contrasts[0]

if model_n == [0]:
    contrasts = [('fs', 'T', ['fs'], [1.0]),
                 ('ss', 'T', ['ss'], [1.0]),
                 ('go', 'T', ['go'], [1.0]),
                 ('fs-go', 'T', ['fs', 'go'], [1, -1]),
                 ('fs-ss', 'T', ['fs', 'ss'], [1, -1]),
                 ('ss-go', 'T', ['ss', 'go'], [1, -1])
                 ]
    
elif model_n == [1]:
    contrasts = [('response_left', 'T', ['response_left'], [1.0]),
                 ('response_right', 'T', ['response_right'], [1.0]),
                 ('left-right', 'T', ['response_left','response_right'], [1,-1])
                 ]

In [6]:
def get_runs_per_sub(subject_id):
    # here we can add the runs per subject if some subject miss a run or two
    if not subject_id in ['11','12']:
        runs = [1,2,3]
    else:
        runs = [1,2]
    
    return runs

In [ ]:
workflow = pe.Workflow(name='feat_level12_sst_roi')
workflow.base_dir = os.path.join(base_dir, 'processing', 'nipype_workflow_folders', this_dataset)
workflow.config = {"execution": {"crashdump_dir":os.path.join(base_dir, 'processing', 'crashdumps')}}

# identity
identity = pe.Node(util.IdentityInterface(fields=['subject_id', 'space', 'model_n']), name='identity')
identity.iterables = [('subject_id', subject_ids),
                      ('space', spaces),
                      ('model_n', model_n)]

# selector
# if running only one run, copy the mask from run 1 to run 2, 'mask' variable below needs a list, it will crash if only one is found
#(terrible way to get around this.. fix at some point.. lazy..)
templates = {'roi_funcs': os.path.join(base_dir, 'derivatives', 'fsl_feat_roi_func', this_dataset,
                                            'sub-{subject_id}', 'func',  
                                            f'sub-{{subject_id}}_task-{task}_run-*_space-{{space}}*_desc-preproc_bold.nii.gz'),
             'mask': os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset, 
                                  'sub-{subject_id}', 'func', 
                                  f'sub-{{subject_id}}_task-{task}_run-*_space-{{space}}_desc-brain_mask.nii.gz')}

selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')
workflow.connect(identity, 'space', selector, 'space')

# get run info per sub
run_info_getter = pe.Node(util.Function(function=get_runs_per_sub,
                                        input_names=['subject_id'],
                                        output_names=['runs']), name='run_info_getter')
workflow.connect(identity, 'subject_id', run_info_getter, 'subject_id')

# session info getter
session_info_getter = pe.MapNode(util.Function(function=get_session_info,
                                     input_names=['subject_id', 'run', 'task', 'this_dataset', 'space', 'shift', 'model_n'],
                                     output_names=['session_info']),
                                 iterfield=['run'],
                                 name='session_info_getter')
session_info_getter.inputs.task=task
session_info_getter.inputs.this_dataset=this_dataset
session_info_getter.inputs.space='MNI152NLin2009cAsym'
session_info_getter.inputs.shift = -t_r/2

workflow.connect(identity, 'subject_id', session_info_getter, 'subject_id')
workflow.connect(run_info_getter, 'runs', session_info_getter, 'run')
workflow.connect(identity, 'model_n', session_info_getter, 'model_n')


# model setup
specifymodel = pe.Node(model.SpecifyModel(), name='specifymodel1')
specifymodel.inputs.input_units = 'secs'
specifymodel.inputs.time_repetition = t_r
specifymodel.inputs.high_pass_filter_cutoff = hpcutoff

workflow.connect(session_info_getter, 'session_info', specifymodel, 'subject_info')
## old flow (with cosines): immediately connect to specifymodel
workflow.connect(selector, 'roi_funcs', specifymodel, 'functional_runs')

# Level 1 design
level1design = pe.Node(interface=fsl.Level1Design(), name="level1design")
level1design.inputs.interscan_interval = t_r
level1design.inputs.bases = {'dgamma': {'derivs': True}}
level1design.inputs.contrasts = contrasts
level1design.inputs.model_serial_correlations = True

workflow.connect(specifymodel, 'session_info', level1design, 'session_info')
# workflow.connect(session_info_getter, 'contrasts', level1design, 'contrasts')

# FEAT model
modelgen = pe.MapNode(interface=fsl.FEATModel(), iterfield=['ev_files', 'fsf_file'], name='modelgen') #, 'args'

workflow.connect(level1design, 'ev_files', modelgen, 'ev_files')
workflow.connect(level1design, 'fsf_files', modelgen, 'fsf_file')
# workflow.connect(session_info_getter, 'confounds', modelgen, 'args')   # add confounds here


# FILM GLS
iterfield = ['design_file', 'in_file', 'tcon_file']
modelestimate = pe.MapNode(interface=fsl.FILMGLS(smooth_autocorr=False,
#                                                  autocorr_noestimate=True,
                                                 mask_size = 0,
                                                 threshold = 10.),  # 
                                                 name='modelestimate',
                                                 iterfield=iterfield,
                                                 mem_gb=10,
                                                 full_data = True)

# ## old flow (with cosines): immediately connect to modelestimate
workflow.connect(selector, 'roi_funcs', modelestimate, 'in_file')
# # new flow: connect high-passed data
# workflow.connect(highpass, 'out_file', modelestimate, 'in_file')


workflow.connect(modelgen, 'design_file', modelestimate, 'design_file')
workflow.connect(modelgen, 'con_file', modelestimate, 'tcon_file')


#### Fixed effects
# merge copes, varcopes
copemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="copemerge")

varcopemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="varcopemerge")

maskemerge = pe.MapNode(interface=fsl.Merge(dimension='t'),
                       iterfield=['in_files'],
                       name="maskemerge")

def sort_copes(files):
    numelements = len(files[0])
    outfiles = []
    for i in range(numelements):
        outfiles.insert(i,[])
        for j, elements in enumerate(files):
            outfiles[i].append(elements[i])
    return outfiles

workflow.connect(modelestimate, ('copes',sort_copes), copemerge, 'in_files')
workflow.connect(modelestimate, ('varcopes',sort_copes), varcopemerge, 'in_files')


level2model = pe.Node(interface=fsl.L2Model(), name='l2model')
def num_copes(files):
    return len(files)
workflow.connect(modelestimate, ('copes',num_copes), level2model, 'num_copes')


pickfirst = lambda x: x[0]
flameo = pe.MapNode(
    interface=fsl.FLAMEO(run_mode='fe'),
    name="flameo",
    iterfield=['cope_file', 'var_cope_file'])

workflow.connect([
    (selector, flameo, [(('mask', pickfirst), 'mask_file')]),
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level2model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

# ## Warp to MNI-space
# def warp_files(copes, varcopes, zstats, tdofs, mat, template_brain):
#     # stolen from https://dpaniukov.github.io/2016/07/14/three-level-analysis-with-fsl-and-ants-2.html
#     import nipype.interfaces.ants as ants

#     out_copes = []
#     out_varcopes = []
#     out_zstats = []
#     out_tdofs = []
    
#     warp = ants.ApplyTransforms()
#     warp.inputs.input_image_type = 0
#     warp.inputs.interpolation = 'Linear'
#     warp.inputs.invert_transform_flags = [False] #,False]
#     warp.inputs.reference_image = template_brain
#     warp.inputs.transforms = mat

#     if not isinstance(copes, list):
#         copes = [copes]
#         varcopes = [varcopes]
#         zstats = [zstats]
#         tdofs = [tdofs]
    
#     for cope in copes:
#         warp.inputs.input_image = cope
#         res=warp.run()
#         out_copes.append(str(res.outputs.output_image))

#     for varcope in varcopes:
#         warp.inputs.input_image = varcope
#         res=warp.run()
#         out_varcopes.append(str(res.outputs.output_image))
        
#     for zstat in zstats:
#         warp.inputs.input_image = zstat
#         res=warp.run()
#         out_zstats.append(str(res.outputs.output_image))
    
#     for tdof in tdofs:
#         warp.inputs.input_image = tdof
#         res=warp.run()
#         out_tdofs.append(str(res.outputs.output_image))

#     return out_copes, out_varcopes, out_zstats, out_tdofs

# warpfunc = pe.MapNode(util.Function(input_names=['copes', 'varcopes', 'zstats', 'tdofs', 'mat','template_brain'],
#                                output_names=['out_copes', 'out_varcopes', 'out_zstats', 'out_tdofs'],
#                                function=warp_files),
#                                iterfield=['copes', 'varcopes', 'zstats', 'tdofs'],
#                   name='warpfunc')

# warpfunc.inputs.template_brain = template_brain
# workflow.connect(flameo, 'copes', warpfunc, 'copes')
# workflow.connect(flameo, 'var_copes', warpfunc, 'varcopes')
# workflow.connect(flameo, 'zstats', warpfunc, 'zstats')
# workflow.connect(flameo, 'tdof', warpfunc, 'tdofs')
# workflow.connect(selector, 'composite', warpfunc, 'mat')

#### post-FE analyis
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')

workflow.connect(selector, ('mask', pickfirst), smoothestimate, 'mask_file')
workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

#
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
workflow.connect(selector, ('mask', pickfirst), get_volume, 'in_file')

# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])] 
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

def convert_volume(input):
    return int(input[1])

workflow.connect(get_volume, ('out_stat', convert_volume), grf_cluster, 'volume')
grf_cluster.inputs.out_threshold_file = True


## datasink
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(base_dir, 'derivatives', 'glm_feat_sst_roi')

#_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_002
substitutions = [(f'mni/level2_{stat_type}s/_model_n_{model_n_}_space_MNI152NLin2009cAsym_subject_id_{sub}/_flameo{contrast_n}/{stat_type}1.nii.gz',
                  f'{this_dataset}/sub-{sub}/func/model-{model_n_}/sub-{sub}_task-{task}_space-MNI152NLin2009cAsym_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
                  for sub in subject_ids
                  for contrast_n in range(len(contrasts))
                  for model_n_ in model_n
                  for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
                  ]


# substitutions = [(f'mni/level2_{stat_type}s/_model_n_{model_n_}_space_MNI152NLin2009cAsym_subject_id_{sub}/_warpfunc{contrast_n}/{stat_type}1_trans.nii.gz',
#                    f'{this_dataset}/sub-{sub}/func/model-{model_n_}/sub-{sub}_task-{task}_space-MNI152NLin2009cAsym_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
#                   for sub in subject_ids
#                   for contrast_n in range(len(contrasts))
#                   for model_n_ in model_n
#                   for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
#                   ]

ds.inputs.substitutions = substitutions


## cluster thresholds: leave for now, this is more of a third-level model thing anyway
# workflow.connect(grf_cluster, 'threshold_file', ds, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'locnalmax_txt_file', ds, 'grf_localmax_txt_file')

## T1w-space
workflow.connect(flameo, 'zstats', ds, 'subject_level_model.mni.level2_zstats')
workflow.connect(flameo, 'copes', ds, 'subject_level_model.mni.level2_copes')
workflow.connect(flameo, 'var_copes', ds, 'subject_level_model.mni.level2_varcopes')
workflow.connect(flameo, 'tdof', ds, 'subject_level_model.mni.level2_tdof_ts')

# ## MNI-space
# workflow.connect(warpfunc, 'out_zstats', ds, 'subject_level_model.mni.level2_zstats')
# workflow.connect(warpfunc, 'out_copes', ds, 'subject_level_model.mni.level2_copes')
# workflow.connect(warpfunc, 'out_varcopes', ds, 'subject_level_model.mni.level2_varcopes')
# workflow.connect(warpfunc, 'out_tdofs', ds, 'subject_level_model.mni.level2_tdof_ts')
# workflow.connect(flameo, 'tdof', ds, 'level2_tdof')

In [8]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs':20, 'memory_gb': 200})

230130-11:41:31,243 nipype.workflow INFO:
	 Workflow feat_level12_sst_roi settings: ['check', 'execution', 'logging', 'monitoring']
230130-11:41:31,577 nipype.workflow INFO:
	 Running in parallel.
230130-11:41:32,465 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 28 jobs ready. Free memory (GB): 200.00/200.00, Free processors: 20/20.
230130-11:41:32,557 nipype.workflow INFO:
	 [Job 1] Cached (feat_level12_sst_roi.run_info_getter).
230130-11:41:32,557 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_15/selector".
230130-11:41:32,562 nipype.workflow INFO:
	 [Job 3] Cached (feat_level12_sst_roi.run_info_getter).
230130-11:41:32,563 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_

230130-11:41:34,536 nipype.workflow INFO:
	 [Node] Executing "selector" <nipype.interfaces.io.SelectFiles>
230130-11:41:34,537 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_09/selector".
230130-11:41:34,538 nipype.workflow INFO:
	 [Node] Executing "selector" <nipype.interfaces.io.SelectFiles>
230130-11:41:34,540 nipype.workflow INFO:
	 [Job 28] Cached (feat_level12_sst_roi.get_volume).
230130-11:41:34,540 nipype.workflow INFO:
	 [Node] Executing "selector" <nipype.interfaces.io.SelectFiles>
230130-11:41:34,540 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.001436s.
230130-11:41:34,541 nipype.workflow INFO:
	 [Node] Executing "selector" <nipype.interfaces.io.SelectFiles>
230130-11:41:34,543 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.001418s.
230130-11:41:34,54

230130-11:41:40,546 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.specifymodel1".
230130-11:41:40,548 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.specifymodel1".
230130-11:41:40,550 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.specifymodel1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_13/specifymodel1".
230130-11:41:40,552 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.specifymodel1".
230130-11:41:40,554 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.specifymodel1".
230130-11:41:40,556 nipype.workflow INFO:
	 [Job 255] Cached (_session_info_getter0).
230130-11:41:40,557 nipype.workflow INFO:
	 [Job 256] Cached (_session_info_getter1).
230130-11:41:40,557 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.specifymode

230130-11:41:42,598 nipype.workflow INFO:
	 [Node] Finished "specifymodel1", elapsed time 0.002914s.
230130-11:41:42,599 nipype.workflow INFO:
	 [Node] Executing "specifymodel1" <nipype.algorithms.modelgen.SpecifyModel>
230130-11:41:42,602 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.level1design" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_14/level1design".
230130-11:41:42,602 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.level1design".
230130-11:41:42,604 nipype.workflow INFO:
	 [Node] Finished "specifymodel1", elapsed time 0.003653s.
230130-11:41:42,608 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.level1design".
230130-11:41:42,606 nipype.workflow INFO:
	 [Node] Executing "specifymodel1" <nipype.algorithms.modelgen.SpecifyModel>
230130-11:41:42,606 nipype.workflow INFO:
	 [Node] Executin

230130-11:41:44,585 nipype.workflow INFO:
	 [Node] Executing "level1design" <nipype.interfaces.fsl.model.Level1Design>
230130-11:41:44,587 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.level1design" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_11/level1design".
230130-11:41:44,588 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.level1design".
230130-11:41:44,595 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.level1design".
230130-11:41:44,598 nipype.workflow INFO:
	 [Node] Executing "level1design" <nipype.interfaces.fsl.model.Level1Design>
230130-11:41:44,600 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.level1design" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_06/l

...

230130-11:54:43,247 nipype.workflow INFO:
	 [Job 829] Completed (_grf_cluster5).
230130-11:54:43,249 nipype.workflow INFO:
	 [Job 830] Completed (_grf_cluster0).
230130-11:54:43,251 nipype.workflow INFO:
	 [MultiProc] Running 6 tasks, and 7 jobs ready. Free memory (GB): 198.80/200.00, Free processors: 14/20.
                     Currently running:
                       * _smoothestimate5
                       * _smoothestimate4
                       * _smoothestimate3
                       * _smoothestimate2
                       * _smoothestimate1
                       * _smoothestimate0
230130-11:54:43,313 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.grf_cluster".
230130-11:54:43,317 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.grf_cluster".
230130-11:54:43,317 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level12_sst_roi.grf_cluster".
230130-11:54:43,319 nipype.workflow INFO:
	 [Node] Outdated ca

230130-11:54:43,338 nipype.workflow INFO:
	 [Node] "_grf_cluster3" found cached.
230130-11:54:43,338 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster5" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_07/_threshold_3.1/grf_cluster/mapflow/_grf_cluster5".
230130-11:54:43,339 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster4" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_09/_threshold_2.3/grf_cluster/mapflow/_grf_cluster4".
230130-11:54:43,339 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster5" - collecting precomputed outputs
230130-11:54:43,339 nipype.workflow INFO:
	 [Node] "_grf_cluster5" found cached.
230130-11:54:43,340 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster4" - collecting precomputed outputs
230130-11:54:43,340 nipype.workflow INFO:


230130-11:54:49,312 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_03/smoothestimate/mapflow/_smoothestimate0".
230130-11:54:49,316 nipype.workflow INFO:
	 [Node] Cached "_smoothestimate0" - collecting precomputed outputs
230130-11:54:49,317 nipype.workflow INFO:
	 [Node] "_smoothestimate0" found cached.
230130-11:54:49,318 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_03/smoothestimate/mapflow/_smoothestimate1".
230130-11:54:49,320 nipype.workflow INFO:
	 [Node] Cached "_smoothestimate1" - collecting precomputed outputs
230130-11:54:49,321 nipype.workflow INFO:
	 [Node] "_smoothestimate1" found cached.
230130-11:54:49,323 nipype.workflow INFO:


230130-11:54:53,322 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster1" <nipype.interfaces.fsl.model.Cluster>
230130-11:54:53,324 nipype.workflow INFO:
	 [Node] Outdated cache found for "_grf_cluster4".
230130-11:54:53,322 nipype.workflow INFO:
	 [Node] Outdated cache found for "_grf_cluster2".
230130-11:54:53,322 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster0" <nipype.interfaces.fsl.model.Cluster>
230130-11:54:53,323 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster3" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_03/_threshold_3.1/grf_cluster/mapflow/_grf_cluster3".
230130-11:54:53,324 nipype.workflow INFO:
	 [Node] Outdated cache found for "_grf_cluster3".
230130-11:54:53,325 nipype.workflow INFO:
	 [Node] Outdated cache found for "_grf_cluster5".
230130-11:54:53,325 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster2" <nipype.interfaces.fsl.model.C

230130-11:54:55,342 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster3" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_03/_threshold_3.1/grf_cluster/mapflow/_grf_cluster3".
230130-11:54:55,343 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster4" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_MNI152NLin2009cAsym_subject_id_03/_threshold_2.3/grf_cluster/mapflow/_grf_cluster4".
230130-11:54:55,343 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster3" - collecting precomputed outputs
230130-11:54:55,344 nipype.workflow INFO:
	 [Node] "_grf_cluster3" found cached.
230130-11:54:55,344 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster4" - collecting precomputed outputs
230130-11:54:55,345 nipype.workflow INFO:
	 [Node] "_grf_cluster4" found cached.
230130-11:54:55,345 nipype.workflow INFO:


## pipeline setup

In [2]:
# Which dataset are we looking at?
# this_dataset = 'Leipzig_7T_SM'
# this_dataset = 'Leipzig_7T_GdH'
# this_dataset = 'NTNU_7T_SJSI'
this_dataset = 'aron_3T'

In [3]:
# general set-up
base_dir = '/home/scotti/projects/3t_7t_sst_comparison'
work_dir = os.path.join(base_dir, 'processing', 'nipype_workflow_folders')
roi_hp_dir = os.path.join(base_dir, 'derivatives', 'fsl_feat_roi_func', this_dataset)

hpcutoff = 10000.   # in seconds
model_n = [0]
spaces = ['T1w']   # shouldn't touch this but just in case we _do_ want to go back to MNI....

if this_dataset == 'Leipzig_7T_SM':
    t_r = 3.0
    numsubs = 18
    subject_ids = [x.split('/')[-2].split('-')[-1] for x in glob.glob(os.path.join(roi_hp_dir, 'sub-*/'))]
    task = 'stop'
elif this_dataset == 'Leipzig_7T_GdH':
    t_r = 2.0
    numsubs = 20
    subject_ids = [x.split('/')[-1].split('-')[-1] for x in glob.glob(os.path.join(roi_hp_dir, 'sub-*T'))]
    task = 'stop'
elif this_dataset == 'aron_3T':
    t_r = 2.0
    subject_ids = [x.split('/')[-2].split('-')[-1] for x in glob.glob(os.path.join(roi_hp_dir, 'sub-*/'))]
    numsubs = len(subject_ids)
    task = 'stopsignal'
    
# make sure all files required are found correctly    
rois_ = glob.glob(os.path.join(base_dir, 'derivatives', 'fsl_feat_roi_func', this_dataset,
                                        'sub-*', 'func',  f'sub-*_task-{task}_run-*_space-*_desc-preproc_bold.nii.gz'))
masks_ = glob.glob(os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset, 
                              'sub-*', 'func', f'sub-*_task-{task}_run-*_space-*_desc-brain_mask.nii.gz'))
comp_ = glob.glob(os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset, 
                                    'sub-*', 'anat', 'sub-*_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'))
xfm_ = glob.glob(os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset,
                              'sub-*', 'func',  f'sub-*_task-{task}_run-*_from-scanner_to-T1w_mode-image_xfm.txt'))
events_ = glob.glob(os.path.join(base_dir, f'derivatives/event_files/{this_dataset}/sub-*/func/sub-*run-*_events.tsv'))
    
template_brain = os.path.join(base_dir,'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')

In [4]:
print(f"""
this dataset : {this_dataset}
tr : {t_r}
number of subjects : {numsubs}
subject ids : {subject_ids}
roi_hp_dir : {roi_hp_dir}
model ns : {model_n}
no. rois : {len(rois_)}
no. masks : {len(masks_)}
no. comps : {len(comp_)}
no. xfms : {len(xfm_)}
no.events : {len(events_)}

""")


this dataset : aron_3T
tr : 2.0
number of subjects : 14
subject ids : ['15', '02', '14', '13', '10', '12', '05', '11', '06', '04', '01', '03', '07', '09']
roi_hp_dir : /home/scotti/projects/3t_7t_sst_comparison/derivatives/fsl_feat_roi_func/aron_3T
model ns : [0]
no. rois : 40
no. masks : 40
no. comps : 14
no. xfms : 40
no.events : 40




# first & second level models

In [5]:
def get_session_info(subject_id, run, task='stop', this_dataset='Leipzig_7T_SM', space='T1w', shift=-1.38/2, model_n=0,
                    root_dir='/home/scotti/projects/3t_7t_sst_comparison', include_physio=True):
    # other space: MNI152NLin2009cAsym
    import pandas as pd
    import numpy as np
    import os
    from nipype.interfaces.base import Bunch
            
    ### files ###
    sub = subject_id
    event_fn = f'{root_dir}/derivatives/event_files/{this_dataset}/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_events.tsv'
    confounds_fn = f'{root_dir}/derivatives/fmriprep/{this_dataset}/fmriprep/fmriprep/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_desc-confounds_timeseries.tsv'

    events = pd.read_csv(event_fn, sep='\t', index_col=None)
    events['duration'] = 0.001  # stick functions
    if model_n == 0:
        events = events.loc[events.trial_type.isin(['ss', 'fs', 'go'])]
    elif model_n == 1:
        events = events.loc[events.trial_type.isin(['response_left', 'response_right'])]
    events = events[['onset', 'trial_type', 'duration']]

    # slice time correction, nb: shift should be a negative number for STC
    events['onset'] += shift  

    ### confounds ###
    confounds = pd.read_csv(confounds_fn, sep='\t')
#        cosine_cols = [x for x in confounds.columns if 'cos' in x]
    include_confounds = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'dvars', 'framewise_displacement'] #+ cosine_cols
    confounds = confounds[include_confounds].fillna(method='bfill')

    # get retroicor
    if include_physio:
        ## take first 20 aCompCor components
#         print("No retroicor found, including 20 a_comp_cor components")
        a_comp_cor = pd.read_csv(confounds_fn, sep='\t')[['a_comp_cor_' + str(x).zfill(2) for x in range(20)]]
        confounds = pd.concat([confounds, a_comp_cor], axis=1)

    # save the confounds that we actually include in the GLM to csv, no header or index
    confounds_fn = f'{root_dir}/derivatives/confounds/{this_dataset}/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_desc-confounds_timeseries.txt'
    os.makedirs(os.path.dirname(confounds_fn), exist_ok=True)
    confounds.to_csv(confounds_fn, sep='\t', header=False, index=False)

    ### Make bunch ###
    if model_n == 0:
        conditions=['fs',
                    'ss',
                    'go']

    elif model_n ==1:
        conditions = [
                     'response_left',
                     'response_right'
                     ]

    onsets = [events.loc[events.trial_type==trial_type,'onset'].tolist() for trial_type in conditions]
    durations = [events.loc[events.trial_type==trial_type,'duration'].tolist() for trial_type in conditions]
    amplitudes = [[1] * events.loc[events.trial_type==trial_type,'onset'].shape[0] for trial_type in conditions]
#             amplitudes = [events.loc[events.trial_type==trial_type,'modulation'].tolist() for trial_type in conditions]

    info = Bunch(conditions=conditions,
                 onsets=onsets,
                 durations=durations,
                 amplitudes=amplitudes)
#         print(info)
    
    # always return info and the confounds_fn
    return info, confounds_fn #, contrasts[0]

if model_n == [0]:
    contrasts = [('fs', 'T', ['fs'], [1.0]),
                 ('ss', 'T', ['ss'], [1.0]),
                 ('go', 'T', ['go'], [1.0]),
                 ('fs-go', 'T', ['fs', 'go'], [1, -1]),
                 ('fs-ss', 'T', ['fs', 'ss'], [1, -1]),
                 ('ss-go', 'T', ['ss', 'go'], [1, -1])
                 ]
    
elif model_n == [1]:
    contrasts = [('response_left', 'T', ['response_left'], [1.0]),
                 ('response_right', 'T', ['response_right'], [1.0]),
                 ('left-right', 'T', ['response_left','response_right'], [1,-1])
                 ]

In [6]:
def get_runs_per_sub(subject_id):
    # here we can add the runs per subject if some subject miss a run or two
    if not subject_id in ['11','12']:
        runs = [1,2,3]
    else:
        runs = [1,2]
    
    return runs

In [7]:
workflow = pe.Workflow(name='feat_level12_sst_roi')
workflow.base_dir = os.path.join(base_dir, 'processing', 'nipype_workflow_folders', this_dataset)
workflow.config = {"execution": {"crashdump_dir":os.path.join(base_dir, 'processing', 'crashdumps')}}

# identity
identity = pe.Node(util.IdentityInterface(fields=['subject_id', 'space', 'model_n']), name='identity')
identity.iterables = [('subject_id', subject_ids),
                      ('space', spaces),
                      ('model_n', model_n)]

# selector
# if running only one run, copy the mask from run 1 to run 2, 'mask' variable below needs a list, it will crash if only one is found
#(terrible way to get around this.. fix at some point.. lazy..)
templates = {'roi_funcs': os.path.join(base_dir, 'derivatives', 'fsl_feat_roi_func', this_dataset,
                                            'sub-{subject_id}', 'func',  
                                            f'sub-{{subject_id}}_task-{task}_run-*_space-{{space}}*_desc-preproc_bold.nii.gz'),
             'mask': os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset, 
                                  'sub-{subject_id}', 'func', 
                                  f'sub-{{subject_id}}_task-{task}_run-*_space-{{space}}_desc-brain_mask.nii.gz'),
             'composite':  os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset, 
                                        'sub-{subject_id}', 'anat', 
                                        'sub-{subject_id}_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'),
             'xfm': os.path.join(base_dir, 'derivatives', 'fmriprep_feat_hack', this_dataset, 
                                  'sub-{subject_id}', 'func', 
                                  f'sub-{{subject_id}}_task-{task}_run-*_from-scanner_to-T1w_mode-image_xfm.txt'),}
selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')
workflow.connect(identity, 'space', selector, 'space')

# get run info per sub
run_info_getter = pe.Node(util.Function(function=get_runs_per_sub,
                                        input_names=['subject_id'],
                                        output_names=['runs']), name='run_info_getter')
workflow.connect(identity, 'subject_id', run_info_getter, 'subject_id')

# session info getter
session_info_getter = pe.MapNode(util.Function(function=get_session_info,
                                     input_names=['subject_id', 'run', 'task', 'this_dataset', 'space', 'shift', 'model_n'],
                                     output_names=['session_info', 'confounds']),
                                 iterfield=['run'],
                                 name='session_info_getter')
session_info_getter.inputs.task=task
session_info_getter.inputs.this_dataset=this_dataset
session_info_getter.inputs.space='T1w'
session_info_getter.inputs.shift = -t_r/2

workflow.connect(identity, 'subject_id', session_info_getter, 'subject_id')
workflow.connect(run_info_getter, 'runs', session_info_getter, 'run')
workflow.connect(identity, 'model_n', session_info_getter, 'model_n')


# model setup
specifymodel = pe.Node(model.SpecifyModel(), name='specifymodel1')
specifymodel.inputs.input_units = 'secs'
specifymodel.inputs.time_repetition = t_r
specifymodel.inputs.high_pass_filter_cutoff = hpcutoff

workflow.connect(session_info_getter, 'session_info', specifymodel, 'subject_info')
## old flow (with cosines): immediately connect to specifymodel
workflow.connect(selector, 'roi_funcs', specifymodel, 'functional_runs')

# Level 1 design
level1design = pe.Node(interface=fsl.Level1Design(), name="level1design")
level1design.inputs.interscan_interval = t_r
level1design.inputs.bases = {'dgamma': {'derivs': True}}
level1design.inputs.contrasts = contrasts
level1design.inputs.model_serial_correlations = True

workflow.connect(specifymodel, 'session_info', level1design, 'session_info')
# workflow.connect(session_info_getter, 'contrasts', level1design, 'contrasts')

# FEAT model
modelgen = pe.MapNode(interface=fsl.FEATModel(), iterfield=['ev_files', 'fsf_file', 'args'], name='modelgen')

workflow.connect(level1design, 'ev_files', modelgen, 'ev_files')
workflow.connect(level1design, 'fsf_files', modelgen, 'fsf_file')
workflow.connect(session_info_getter, 'confounds', modelgen, 'args')   # add confounds here


# FILM GLS
iterfield = ['design_file', 'in_file', 'tcon_file']
modelestimate = pe.MapNode(interface=fsl.FILMGLS(smooth_autocorr=False,
                                                 autocorr_noestimate=True,
                                                 mask_size = 0,
                                                 threshold = 10.0),  # 
                                                 name='modelestimate',
                                                 iterfield=iterfield,
                                                 mem_gb=10)

# ## old flow (with cosines): immediately connect to modelestimate
workflow.connect(selector, 'roi_funcs', modelestimate, 'in_file')
# # new flow: connect high-passed data
# workflow.connect(highpass, 'out_file', modelestimate, 'in_file')


workflow.connect(modelgen, 'design_file', modelestimate, 'design_file')
workflow.connect(modelgen, 'con_file', modelestimate, 'tcon_file')


#### Fixed effects
# merge copes, varcopes
copemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="copemerge")

varcopemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="varcopemerge")

maskemerge = pe.MapNode(interface=fsl.Merge(dimension='t'),
                       iterfield=['in_files'],
                       name="maskemerge")

def sort_copes(files):
    numelements = len(files[0])
    outfiles = []
    for i in range(numelements):
        outfiles.insert(i,[])
        for j, elements in enumerate(files):
            outfiles[i].append(elements[i])
    return outfiles

workflow.connect(modelestimate, ('copes',sort_copes), copemerge, 'in_files')
workflow.connect(modelestimate, ('varcopes',sort_copes), varcopemerge, 'in_files')


level2model = pe.Node(interface=fsl.L2Model(), name='l2model')
def num_copes(files):
    return len(files)
workflow.connect(modelestimate, ('copes',num_copes), level2model, 'num_copes')


pickfirst = lambda x: x[0]
flameo = pe.MapNode(
    interface=fsl.FLAMEO(run_mode='fe'),
    name="flameo",
    iterfield=['cope_file', 'var_cope_file'])

workflow.connect([
    (selector, flameo, [(('mask', pickfirst), 'mask_file')]),
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level2model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

## Warp to MNI-space
def warp_files(copes, varcopes, zstats, tdofs, mat, template_brain):
    # stolen from https://dpaniukov.github.io/2016/07/14/three-level-analysis-with-fsl-and-ants-2.html
    import nipype.interfaces.ants as ants

    out_copes = []
    out_varcopes = []
    out_zstats = []
    out_tdofs = []
    
    warp = ants.ApplyTransforms()
    warp.inputs.input_image_type = 0
    warp.inputs.interpolation = 'Linear'
    warp.inputs.invert_transform_flags = [False] #,False]
    warp.inputs.reference_image = template_brain
    warp.inputs.transforms = mat

    if not isinstance(copes, list):
        copes = [copes]
        varcopes = [varcopes]
        zstats = [zstats]
        tdofs = [tdofs]
    
    for cope in copes:
        warp.inputs.input_image = cope
        res=warp.run()
        out_copes.append(str(res.outputs.output_image))

    for varcope in varcopes:
        warp.inputs.input_image = varcope
        res=warp.run()
        out_varcopes.append(str(res.outputs.output_image))
        
    for zstat in zstats:
        warp.inputs.input_image = zstat
        res=warp.run()
        out_zstats.append(str(res.outputs.output_image))
    
    for tdof in tdofs:
        warp.inputs.input_image = tdof
        res=warp.run()
        out_tdofs.append(str(res.outputs.output_image))

    return out_copes, out_varcopes, out_zstats, out_tdofs

warpfunc = pe.MapNode(util.Function(input_names=['copes', 'varcopes', 'zstats', 'tdofs', 'mat','template_brain'],
                               output_names=['out_copes', 'out_varcopes', 'out_zstats', 'out_tdofs'],
                               function=warp_files),
                               iterfield=['copes', 'varcopes', 'zstats', 'tdofs'],
                  name='warpfunc')

warpfunc.inputs.template_brain = template_brain
workflow.connect(flameo, 'copes', warpfunc, 'copes')
workflow.connect(flameo, 'var_copes', warpfunc, 'varcopes')
workflow.connect(flameo, 'zstats', warpfunc, 'zstats')
workflow.connect(flameo, 'tdof', warpfunc, 'tdofs')
workflow.connect(selector, 'composite', warpfunc, 'mat')

#### post-FE analyis
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')

workflow.connect(selector, ('mask', pickfirst), smoothestimate, 'mask_file')
workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

#
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
workflow.connect(selector, ('mask', pickfirst), get_volume, 'in_file')

# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])] 
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

def convert_volume(input):
    return int(input[1])

workflow.connect(get_volume, ('out_stat', convert_volume), grf_cluster, 'volume')
grf_cluster.inputs.out_threshold_file = True


## datasink
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(base_dir, 'derivatives', 'glm_feat_sst_roi')

#_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_002
substitutions = [(f't1w/level2_{stat_type}s/_model_n_{model_n_}_space_T1w_subject_id_{sub}/_flameo{contrast_n}/{stat_type}1.nii.gz',
                  f'{this_dataset}/sub-{sub}/func/model-{model_n_}/sub-{sub}_task-{task}_space-T1w_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
                  for sub in subject_ids
                  for contrast_n in range(len(contrasts))
                  for model_n_ in model_n
                  for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
                  ]


substitutions += [(f'mni/level2_{stat_type}s/_model_n_{model_n_}_space_T1w_subject_id_{sub}/_warpfunc{contrast_n}/{stat_type}1_trans.nii.gz',
                   f'{this_dataset}/sub-{sub}/func/model-{model_n_}/sub-{sub}_task-{task}_space-MNI152NLin2009cAsym_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
                  for sub in subject_ids
                  for contrast_n in range(len(contrasts))
                  for model_n_ in model_n
                  for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
                  ]

ds.inputs.substitutions = substitutions


## cluster thresholds: leave for now, this is more of a third-level model thing anyway
# workflow.connect(grf_cluster, 'threshold_file', ds, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'locnalmax_txt_file', ds, 'grf_localmax_txt_file')

## T1w-space
workflow.connect(flameo, 'zstats', ds, 'subject_level_model.t1w.level2_zstats')
workflow.connect(flameo, 'copes', ds, 'subject_level_model.t1w.level2_copes')
workflow.connect(flameo, 'var_copes', ds, 'subject_level_model.t1w.level2_varcopes')
workflow.connect(flameo, 'tdof', ds, 'subject_level_model.t1w.level2_tdof_ts')

## MNI-space
workflow.connect(warpfunc, 'out_zstats', ds, 'subject_level_model.mni.level2_zstats')
workflow.connect(warpfunc, 'out_copes', ds, 'subject_level_model.mni.level2_copes')
workflow.connect(warpfunc, 'out_varcopes', ds, 'subject_level_model.mni.level2_varcopes')
workflow.connect(warpfunc, 'out_tdofs', ds, 'subject_level_model.mni.level2_tdof_ts')
# workflow.connect(flameo, 'tdof', ds, 'level2_tdof')

In [9]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs':20, 'memory_gb': 200})

221202-14:45:43,766 nipype.workflow INFO:
	 Workflow feat_level12_sst_roi settings: ['check', 'execution', 'logging', 'monitoring']
221202-14:45:43,915 nipype.workflow INFO:
	 Running in parallel.
221202-14:45:43,918 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 28 jobs ready. Free memory (GB): 200.00/200.00, Free processors: 20/20.
221202-14:45:44,11 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_T1w_subject_id_15/selector".
221202-14:45:44,12 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.run_info_getter" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_T1w_subject_id_15/run_info_getter".
221202-14:45:44,12 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_roi.selector" in "/home/scotti/projects/3t_7t_sst_comparison/

221202-14:45:44,23 nipype.workflow INFO:
	 [Node] Executing "run_info_getter" <nipype.interfaces.utility.wrappers.Function>
221202-14:45:44,23 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.001822s.
221202-14:45:44,25 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.000482s.
221202-14:45:44,25 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.000591s.
221202-14:45:44,26 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.000541s.
221202-14:45:44,26 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.001792s.
221202-14:45:44,26 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.000581s.
221202-14:45:44,27 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.002104s.
221202-14:45:44,27 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.002187s.
221202-14:45:44,27 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.00

221202-14:45:48,36 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_T1w_subject_id_15/session_info_getter/mapflow/_session_info_getter0".
221202-14:45:48,36 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_T1w_subject_id_15/session_info_getter/mapflow/_session_info_getter1".
221202-14:45:48,37 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter2" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_T1w_subject_id_15/session_info_getter/mapflow/_session_info_getter2".
221202-14:45:48,37 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_w

No retroicor found, including 20 a_comp_cor componentsNo retroicor found, including 20 a_comp_cor components

No retroicor found, including 20 a_comp_cor componentsNo retroicor found, including 20 a_comp_cor componentsNo retroicor found, including 20 a_comp_cor components

No retroicor found, including 20 a_comp_cor componentsNo retroicor found, including 20 a_comp_cor components
No retroicor found, including 20 a_comp_cor components


No retroicor found, including 20 a_comp_cor componentsNo retroicor found, including 20 a_comp_cor componentsNo retroicor found, including 20 a_comp_cor components
No retroicor found, including 20 a_comp_cor components


No retroicor found, including 20 a_comp_cor componentsNo retroicor found, including 20 a_comp_cor componentsNo retroicor found, including 20 a_comp_cor components


No retroicor found, including 20 a_comp_cor componentsNo retroicor found, including 20 a_comp_cor components

221202-14:45:49,434 nipype.workflow INFO:
	 [Node] Finished "_ses

221202-14:45:49,994 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter2" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_T1w_subject_id_04/session_info_getter/mapflow/_session_info_getter2".
221202-14:45:49,994 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_T1w_subject_id_10/session_info_getter/mapflow/_session_info_getter0".
221202-14:45:49,995 nipype.workflow INFO:
	 [Node] Executing "_session_info_getter1" <nipype.interfaces.utility.wrappers.Function>
221202-14:45:49,995 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_T1w_subject_id_01/session_info_getter/mapflow/_session_info_getter0".
221202-14:45:49,9

...

221202-14:47:16,116 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/t1w/level2_varcopes/_model_n_0_space_T1w_subject_id_07/_flameo1/varcope1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/aron_3T/sub-07/func/model-0/sub-07_task-stopsignal_space-T1w_model-0_contrast-1_desc-varcope.nii.gz
221202-14:47:16,116 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/t1w/level2_varcopes/_model_n_0_space_T1w_subject_id_07/_flameo2/varcope1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/aron_3T/sub-07/func/model-0/sub-07_task-stopsignal_space-T1w_model-0_contrast-2_desc-varcope.nii.gz
221202-14:47:16,117 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/t1w/level2_varcopes/_mod

221202-14:47:16,129 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/mni/level2_copes/_model_n_0_space_T1w_subject_id_07/_warpfunc3/cope1_trans.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/aron_3T/sub-07/func/model-0/sub-07_task-stopsignal_space-MNI152NLin2009cAsym_model-0_contrast-3_desc-cope.nii.gz
221202-14:47:16,130 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/mni/level2_copes/_model_n_0_space_T1w_subject_id_07/_warpfunc4/cope1_trans.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/aron_3T/sub-07/func/model-0/sub-07_task-stopsignal_space-MNI152NLin2009cAsym_model-0_contrast-4_desc-cope.nii.gz
221202-14:47:16,130 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_

221202-14:47:18,84 nipype.workflow INFO:
	 [Node] Cached "_warpfunc1" - collecting precomputed outputs
221202-14:47:18,85 nipype.workflow INFO:
	 [Node] "_warpfunc1" found cached.
221202-14:47:18,86 nipype.workflow INFO:
	 [Node] Setting-up "_warpfunc2" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_T1w_subject_id_09/warpfunc/mapflow/_warpfunc2".
221202-14:47:18,88 nipype.workflow INFO:
	 [Node] Cached "_warpfunc2" - collecting precomputed outputs
221202-14:47:18,88 nipype.workflow INFO:
	 [Node] "_warpfunc2" found cached.
221202-14:47:18,90 nipype.workflow INFO:
	 [Node] Setting-up "_warpfunc3" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_roi/_model_n_0_space_T1w_subject_id_09/warpfunc/mapflow/_warpfunc3".
221202-14:47:18,91 nipype.workflow INFO:
	 [Node] Cached "_warpfunc3" - collecting precomputed outputs
221202-14:47:18,91 nipype.workflow IN

221202-14:47:20,117 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/t1w/level2_varcopes/_model_n_0_space_T1w_subject_id_09/_flameo2/varcope1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/aron_3T/sub-09/func/model-0/sub-09_task-stopsignal_space-T1w_model-0_contrast-2_desc-varcope.nii.gz
221202-14:47:20,118 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/t1w/level2_varcopes/_model_n_0_space_T1w_subject_id_09/_flameo3/varcope1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/aron_3T/sub-09/func/model-0/sub-09_task-stopsignal_space-T1w_model-0_contrast-3_desc-varcope.nii.gz
221202-14:47:20,119 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/t1w/level2_varcopes/_mod

221202-14:47:20,132 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/mni/level2_copes/_model_n_0_space_T1w_subject_id_09/_warpfunc4/cope1_trans.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/aron_3T/sub-09/func/model-0/sub-09_task-stopsignal_space-MNI152NLin2009cAsym_model-0_contrast-4_desc-cope.nii.gz
221202-14:47:20,133 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/mni/level2_copes/_model_n_0_space_T1w_subject_id_09/_warpfunc5/cope1_trans.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/aron_3T/sub-09/func/model-0/sub-09_task-stopsignal_space-MNI152NLin2009cAsym_model-0_contrast-5_desc-cope.nii.gz
221202-14:47:20,133 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_

In [10]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/crashdumps/crash-20221130-134015-scotti-_modelestimate2-c8f14e53-d5b2-4b8e-bd60-f4e47e4384c0.pklz')
res

{'node': _modelestimate2,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 750, in _run_command\n    raise NodeExecutionError(\n',
  'nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node _modelestimate2.\n\nTraceback (most recent call last):\n  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/si

In [8]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/crashdumps/crash-20220725-201456-scotti-_modelgen2-853c3a8d-a5d2-434c-9c99-ac3b0735bd4a.pklz')
res

{'node': _modelgen2,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 750, in _run_command\n    raise NodeExecutionError(\n',
  'nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node _modelgen2.\n\nTraceback (most recent call last):\n  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/interfaces/base/core.py", line 401, in run\n    outputs = sel

ERROR! Session/line number was not unique in database. History logging moved to new session 260


In [24]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_3p125_space_T1w_subject_id_01/modelgen/result_modelgen.pklz')
res

In [27]:
res.outputs

Bunch(con_file=['/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_3p125_space_T1w_subject_id_01/modelgen/mapflow/_modelgen0/run0.con',
       '/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_3p125_space_T1w_subject_id_01/modelgen/mapflow/_modelgen1/run1.con',
       '/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_3p125_space_T1w_subject_id_01/modelgen/mapflow/_modelgen2/run2.con'],
      design_cov=<undefined>,
      design_file=['/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_3p125_space_T1w_subject_id_01/modelgen/mapflow/_modelgen0/run0.mat',
       '/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level12_sst_h

In [3]:
check1 == check2

False